In [41]:
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import os
import re
import random
import itertools
from collections import defaultdict
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from tqdm import tqdm

In [9]:
labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
data = load_dataset(
    "csv",
    data_files={"train": "data/train.csv", "validation": "data/valid.csv"})
data

DatasetDict({
    train: Dataset({
        features: ['text', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral'],
        num_rows: 5426
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")
max_len = 8

data = data.map(lambda examples: tokenizer(examples["text"],
                                           truncation=True,
                                           add_special_tokens=True,
                                           max_length=max_len,
                                           padding="max_length"), batched=True)

def one_hot_to_list(example):
    emotions = []
    for emotion in labels:
        emotions.append(example[emotion])
    example["one_hot_labels"] = emotions

    return example

data = data.map(one_hot_to_list)

- `AutoTokenizer`:  `Универсальный` класс для работы с различными токенизаторами в библиотеке Hugging Face

- `from_pretrained`("ai-forever/ruBert-base"): Загружает предобученный токенизатор, совместимый с моделью ruBert-base

- `max_len`: Максимальная длина последовательности токенов. 
    - `truncation`: Тексты, содержащие больше токенов, будут `обрезаны`. 
    - `padding`="max_length": Короткие тексты будут дополнены `паддингом`(нулями)
- `add_special_tokens`: Добавляет специальные токены `[CLS]` (в начало) и `[SEP]` (в конец)

In [32]:
display(data['train'][0]['input_ids'])
display(data['train'][0]['attention_mask'])
display(data['train'][0]['token_type_ids'])
display(data['train'][0]['one_hot_labels'])

[101, 4137, 29093, 25985, 179, 736, 780, 102]

[1, 1, 1, 1, 1, 1, 1, 1]

[0, 0, 0, 0, 0, 0, 0, 0]

[0, 0, 0, 0, 0, 0, 1]

- `input_ids`: `индексы` токенов, которые берутся из словаря `токенизатора` для подачи в модель

- `attention_mask`: формируется токенизатором автоматически и указывает, какие токены в последовательности являются значимыми (1), а какие были добавлены для заполнения (padding, 0)
- `token_type_ids`: используются для указания модели, к какому предложению (или части текста) относится каждый токен. Это особенно важно для задач, где вход состоит из двух предложений
- `one_hot_labels`: возвращает лейбел как OHE-список
___

In [33]:
class EmotionDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        return {
            'input_ids': torch.tensor(self.dataset[index]["input_ids"], dtype=torch.long),
            'attention_mask': torch.tensor(self.dataset[index]["attention_mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(self.dataset[index]["token_type_ids"], dtype=torch.long),
            'labels': torch.tensor(self.dataset[index]["one_hot_labels"], dtype=torch.float)
        }

Класс `EmotionDataset` служит для `преобразования` токенизированного набора данных в формат, пригодный для использования в `PyTorch DataLoader`

Возвращает необходимые данные в подходящем формате данных - `torch.long` и `torch.float`

In [34]:
train_dataset = EmotionDataset(data["train"])
valid_dataset = EmotionDataset(data["validation"])

train_dataloader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True)
valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=64,
    shuffle=False)

Создание `PyTorch DataLoader` из `EmotionDataset`
___

In [ ]:
class Model(torch.nn.Module):
    def __init__(self, pretrained_model, hidden_dim, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(pretrained_model)
        self.fc = torch.nn.Linear(hidden_dim, num_classes)
        self.dropout = torch.nn.Dropout(p=0.3)

    def forward(self, ids, mask, token_type_ids):
        _, features = self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        features = self.dropout(features)
        output = self.fc(features)
        return output

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model(
    pretrained_model="ai-forever/ruBert-base",
    hidden_dim=768,
    num_classes=len(labels))

model = model.to(device)

- `hidden_dim`: Размерность скрытого слоя

- `ids`: Тензор с токенизированными идентификаторами слов
- `mask`: Маска внимания, чтобы игнорировать padding токены
- `token_type_ids`: Указывает, к какому предложению (в случае двух предложений) принадлежат токены.
- `self.bert`: Возвращает два значения: скрытые состояния для всех токенов (`features`) и скрытое состояние CLS-токена (`_`)
- `return_dict`=False: указывает, что выход модели должен быть в виде `кортежа`, а не словаря.

___

In [36]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.00001, weight_decay=0.)

- `BCEWithLogitsLoss` : Функция потерь, которая объединяет две операции:
    - `Сигмоидная` функция для получения логитов в диапазоне [0, 1]
    - `Binary Cross-Entropy` (BCE):Вычисляет бинарную кросс-энтропию, которая используется для задач многоклассовой классификации

- `weight_decay`:  L2-Регуляризация для предотвращения переобучения.

___

In [43]:
def train(model, criterion, optimizer, dataloader):
    train_loss = 0.0
    model.train()
    for idx, data in enumerate(tqdm(dataloader, desc="Training")):
        ids = data["input_ids"].to(device, dtype=torch.long)
        mask = data["attention_mask"].to(device, dtype=torch.long)
        token_type_ids = data["token_type_ids"].to(device, dtype=torch.long)
        labels = data["labels"].to(device, dtype=torch.float)

        outputs = model(ids, mask, token_type_ids)
        loss = criterion(outputs, labels)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f'Train loss: {train_loss / len(dataloader)}')

    return model

- `model.train()`: включает такие функции, как Dropout, которые должны работать только во `время обучени`

1. `outputs`: логиты, до применения функции активации
2. `loss`: Потери для текущего батча
    - `train_loss`: Общие текущие потери
3. `loss.backward()`: Вычисление градиентов
4. `optimizer.step()`: Обновление параметров модели

In [44]:
def validation(model, criterion, dataloader):
    val_loss = 0.0
    model.eval()
    val_targets, val_outputs = [], []
    with torch.no_grad():
        for idx, data in enumerate(tqdm(dataloader, desc="Valitation")):
            ids = data["input_ids"].to(device, dtype=torch.long)
            mask = data["attention_mask"].to(device, dtype=torch.long)
            token_type_ids = data["token_type_ids"].to(device, dtype=torch.long)
            labels = data["labels"].to(device, dtype=torch.float)

            outputs = model(ids, mask, token_type_ids)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            val_targets.extend(labels.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    print(f'Valid loss: {val_loss / len(dataloader)}')

    return val_outputs, val_targets

- `model.eval()`: отключает механизмы, используемые только во время обучения, например Dropout и BatchNorm, что делает предсказания детерминированными

- `with torch.no_grad()`: Отключение градиентов

In [ ]:
epochs = 3

for epoch in range(epochs):
    print(f"Epoch: {epoch}")
    model = train(model, criterion, optimizer, train_dataloader)
    val_outputs, val_targets = validation(model, criterion, valid_dataloader)

Обучение и расчет функции потерь для обучения и валидации для текущей эпохе

In [50]:
THRESHOLD = 0.3

outputs, targets = validation(model, criterion, valid_dataloader)
outputs = np.array(outputs) >= THRESHOLD

print(metrics.classification_report(targets, outputs, target_names=labels))

Valitation: 100%|██████████| 85/85 [00:30<00:00,  2.75it/s]

Valid loss: 0.42584245380233315
              precision    recall  f1-score   support

       anger       0.10      0.08      0.09       717
     disgust       0.00      0.00      0.00        97
        fear       0.03      0.02      0.02       105
         joy       0.41      1.00      0.58      2219
     sadness       0.07      0.00      0.00       390
    surprise       0.10      0.10      0.10       624
     neutral       0.33      1.00      0.49      1766

   micro avg       0.34      0.69      0.46      5918
   macro avg       0.15      0.31      0.18      5918
weighted avg       0.28      0.69      0.39      5918
 samples avg       0.35      0.71      0.46      5918



`np.array(outputs) >= THRESHOLD`: 
- массив со значениями `1` , если вероятность больше или равна THRESHOLD
- массив со значениями `0` , если вероятность меньше THRESHOLD